<h1> Datto RMM - CSV - Download and Condition </h1>

In [ ]:
# general purpose
import time
import datetime as dt

# Download and Read CSV Information
import re

import numpy as np
import requests
from urllib import request as req

# Spider
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
import re
import lxml

# Data Ingest and Export
import pandas as pd
import csv
import json

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/'

# import configparser for env secrets
from configparser import ConfigParser
config = ConfigParser()
config.read('d:/git/example_infrastructure_data_dev/config/env.ini')
import requests
from requests.structures import CaseInsensitiveDict

# Create Bot Object

## Create Chrome Driver

In [ ]:
chrome_driver = 'd:/web_drivers/chrome/chromedriver_win32/chromedriver.exe'
options = webdriver.ChromeOptions()
options.add_argument('headless')
c_driver = webdriver.Chrome(executable_path=chrome_driver)

## Request Header Info (if needed to impersonate api browser calls)

In [ ]:
# construct header
headers = CaseInsensitiveDict()
headers['User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'

# construct req body
data = CaseInsensitiveDict()
data = ''

In [ ]:
baseUrl = 'https://concord.centrastage.net'
auth_url = f'https://auth.datto.com'
#home_url = f'{baseUrl}'
links = []

# Login to DattoRMM

In [ ]:
c_driver.get(auth_url)

In [ ]:
title = c_driver.find_element_by_xpath('//h1')
title.text

In [ ]:
login_txtbox = c_driver.find_element_by_xpath("//input[@id='form_username']")
login_txtbox.send_keys((config['dattormm']['username']),Keys.ENTER)

In [ ]:
pass_txtbox = c_driver.find_element_by_xpath("//input[@id='form_password']")
pass_txtbox.send_keys((config['dattormm']['password']),Keys.ENTER)

In [ ]:
time.sleep(3)

In [ ]:
mfa_txtbox = c_driver.find_element_by_id('authy_token')
mfa_txtbox.send_keys('453731', Keys.ENTER) # number entered manually for now /TODO: automate MFA

# Non-Compliance Summary Page

In [ ]:
projectUrl = f'{baseUrl}/csm/system/softwaremanagement?sort=&order=&policyid=&c=All&selectItem=All&smstatus=NOT_COMPLIANT&query='
c_driver.get(projectUrl)

In [ ]:
tables = c_driver.find_elements_by_xpath('//table[@class="list"]')

In [ ]:
applications_table = tables[1].find_elements_by_xpath('.//tbody/tr/td')

In [ ]:
iter_col_labels = ['applicationName','verWin32','verWin64','vermacOS','nonComplientCount','blankCol']

pol_iter_counts = len(applications_table)/4

cols_label_iteration = []

for x in range(0,int(pol_iter_counts),1):
    for p in iter_col_labels:
        cols_label_iteration.append(p)

In [ ]:
app_summary_list = []
app_dicts = {}
targets_prog = re.compile(r'[^\,]+')
for label in range(0, len(applications_table)):
    app_dict = {}
    #print(cols_label_iteration[label])
    #print(applications_table[label].text)
    try:

        if cols_label_iteration[label] == 'blankCol':
            app_summary_list.append(app_dicts)
            app_dicts = {}
        elif (cols_label_iteration[label] == 'nonComplientCount') & (applications_table[label].text != 0):
            app_dict[cols_label_iteration[label]] = applications_table[label].text
            h = applications_table[label].find_elements_by_xpath('./a')
            for x in h:
                app_dict['href'] = x.get_attribute('href')

            app_dicts.update(app_dict)
        else:
            #print('-'*100)
            app_dict[cols_label_iteration[label]] = applications_table[label].text
            app_dicts.update(app_dict)
            #app_summary_list.append(app_dict)
    except Exception as e:
        print(e)

In [ ]:
df = pd.DataFrame(app_summary_list)

In [ ]:
df

In [ ]:
href_prog = re.compile(r'(https://.*){1}')

In [ ]:
software_href = []
for element in df['href']:
    try:
        print(href_prog.findall(element))
        software_href.append(element)
    except:
        continue

In [ ]:
https://concord.centrastage.net/csm/account/listDevices?softwaremanagement=[REDACTED_UUID]


In [ ]:
csv_download_prog = re.compile(r'()+')

In [ ]:
list_id = '[REDACTED_UUID]'

In [ ]:
csv_base_url = 'https://concord.centrastage.net/csm/csv/download?&softwaremanagement='
csv_download_template = f'https://concord.centrastage.net/csm/csv/download?&softwaremanagement={list_id}&action=listDevices&controller=account&max=50&offset=0&sort=hostname&order=asc&id=5408'

In [ ]:
print(csv_download_template)

In [ ]:

response = req.urlopen(csv_download_template)
file = response.read()
csv_file = str(file)
print(csv_file)

In [ ]:
download_csv_data(csv_download_template)

In [ ]:
response = req.urlopen(csv_resp)
file = response.read()
file.decode()

In [ ]:
csv_file = file.decode()

In [ ]:
csv_split = csv_file.split('\\n')

In [ ]:
with open('[REDACTED]/.csv','w') as file:
    file.write(csv_split[0])



In [ ]:
stock_info_prog = re.compile(r'([^\n]+)\n?')

In [ ]:
parse = stock_info_prog.findall(csv_split[0])

In [ ]:
parse

In [ ]:
parse_row = []
heading = parse[0].split(',')
for row in parse[1:]:
    element_row = {}
    for index, element in enumerate(row.split(',')):
        element_dict = {}
        element_dict[f'{heading[index]}'] = element
        element_row.update(element_dict)
    parse_row.append(element_row)

In [ ]:
df = pd.DataFrame(parse_row)

In [ ]:
df